En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [1]:
%reload_ext autoreload
%autoreload 2

print("IPython autoreload activated")

IPython autoreload activated


In [2]:
import os

from google.colab import auth
auth.authenticate_user()


In [3]:
from google.cloud import bigquery

# Crear una instancia del cliente
client = bigquery.Client("tw-techdash")

In [4]:
!pip install memray
!pip install memory-profiler



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.2/567.2 kB 26.7 MB/s eta 0:00:00


In [5]:
# Local file paths (consider user input/environment variables for flexibility)
MOUNT_POINT = os.environ['LC_MOUNT_POINT'] = "/content/drive"
SOURCE_PATH = os.environ['LC_SOURCE_PATH'] = "/content/drive/Othercomputers/My MacBook Pro/challenge_DE"


In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
# Mount Google Drive (optional, depending on usage context)
# Uncomment the following line if mounting is needed:
drive.mount(MOUNT_POINT, force_remount=True)

# Change directory using a more explicit method
target_directory = os.path.join(MOUNT_POINT, SOURCE_PATH, "src")
os.chdir(target_directory)

# Confirm the current directory:
print(f"Changed directory to: {os.getcwd()}")  # Use os.getcwd() for accuracy

Mounted at /content/drive
Changed directory to: /content/drive/Othercomputers/My MacBook Pro/challenge_DE/src


In [8]:
query = """
WITH datos AS (
  SELECT
    FORMAT_TIMESTAMP('%Y-%m-%d', TIMESTAMP(date)) AS fecha_formateada,
    user.username as usuario
  FROM
    `tw-techdash.de_challenge_latam.tweets_latam`
),

conteo_por_fecha AS (
  SELECT
    fecha_formateada,
    COUNT(usuario) AS num_tweets
  FROM
    datos
  GROUP BY
    fecha_formateada
  ORDER BY
    num_tweets DESC
  LIMIT 10
),

usuario_top_por_fecha AS (
  SELECT
    fecha_formateada,
    usuario,
    COUNT(usuario) AS num_tweets_por_usuario,
    RANK() OVER (PARTITION BY fecha_formateada ORDER BY COUNT(usuario) DESC) AS rank
  FROM
    datos
  WHERE
    fecha_formateada IN (SELECT fecha_formateada FROM conteo_por_fecha)
  GROUP BY
    fecha_formateada, usuario
)

SELECT
  u.usuario AS top_user,
  u.fecha_formateada as date,
FROM
  usuario_top_por_fecha u
JOIN
  conteo_por_fecha c
ON
  u.fecha_formateada = c.fecha_formateada
WHERE
  u.rank = 1
ORDER BY
  c.num_tweets DESC;

"""

query_job = client.query(query)

query_job.result()

results = query_job.result()

output = [(row.top_user, row.date) for row in results]

print("QUERY 1")
print ("Las top 10 fechas donde hay más tweets. Ordenadas por usuario (username) que más publicaciones tiene por cada uno de esos días: ")
print(output)

QUERY 1
Las top 10 fechas donde hay más tweets. Ordenadas por usuario (username) que más publicaciones tiene por cada uno de esos días: 
[('RanbirS00614606', '2021-02-12'), ('MaanDee08215437', '2021-02-13'), ('RaaJVinderkaur', '2021-02-17'), ('jot__b', '2021-02-16'), ('rebelpacifist', '2021-02-14'), ('neetuanjle_nitu', '2021-02-18'), ('jot__b', '2021-02-15'), ('MangalJ23056160', '2021-02-20'), ('Surrypuria', '2021-02-23'), ('Preetm91', '2021-02-19')]


In [9]:
%run q1_memory.py

q1_memory("")

In [10]:

from memory_profiler import memory_usage
import time

def block_1():
    query = """
    SELECT
      emoji,
      COUNT(*) AS total_emoji
    FROM
      (SELECT
            REGEXP_EXTRACT_ALL(
              content,
              r"[\p{So}]"
            ) AS emojis,
            id
          FROM
            `tw-techdash.de_challenge_latam.tweets_latam`
          WHERE id is not null
    )
    CROSS JOIN UNNEST(emojis) as emoji
    WHERE emoji != '\\uFFFD' -- �
    group by emoji
    order by total_emoji desc
    limit 10
    """

    query_job = client.query(query)

    query_job.result()

    results = query_job.result()

    output = [(row.emoji, row.total_emoji) for row in results]

    print("QUERY 2")
    print("Los emojis más utilizados y la cantidad")
    print(output)

def monitor_memory():

    mem_usage = memory_usage((block_1,))
    print(f"Uso máximo de memoria: {max(mem_usage)} MiB")

start_time = time.time()
monitor_memory()
end_time = time.time()
print(f"Tiempo de ejecución: {end_time - start_time} segundos")


QUERY 2
Los emojis más utilizados y la cantidad
[('✊', 2411), ('❤', 1779), ('☮', 316), ('✌', 274), ('♂', 184), ('♀', 148), ('✍', 103), ('♥', 81), ('⛳', 68), ('✅', 57)]
Uso máximo de memoria: 191.80859375 MiB
Tiempo de ejecución: 1.4669313430786133 segundos


In [11]:

query = """
WITH gruped AS (
  SELECT
    user.username AS username,
    m.username AS top_mentioned,
    COUNT(*) AS cuenta
  FROM
    `tw-techdash.de_challenge_latam.tweets_latam` as tweets,
    UNNEST(mentionedUsers) AS m
  WHERE tweets.id is not null
  GROUP BY
    username, top_mentioned
)

SELECT
  top_mentioned,
  SUM(cuenta) AS total_mencioned
FROM
  gruped
GROUP BY
  top_mentioned
  order by total_mencioned desc
LIMIT 10


"""


query_job = client.query(query)


query_job.result()

results = query_job.result()

output = [(row.top_mentioned, row.total_mencioned) for row in results]

print("QUERY 3")
print ("El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos: ")
print(output)

QUERY 3
El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos: 
[('narendramodi', 2265), ('Kisanektamorcha', 1840), ('RakeshTikaitBKU', 1644), ('PMOIndia', 1427), ('RahulGandhi', 1146), ('GretaThunberg', 1048), ('RaviSinghKA', 1019), ('rihanna', 986), ('UNHumanRights', 962), ('meenaharris', 926)]
